#Using TensorFlow and Keras for Transfer Learning

In [ ]:
#Import Libraries and load a pre-trained model
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import os

Load a model like VGG16 (Without the final classification layer) to use it as a feature extractor.

In [ ]:
#Load the pre-trained model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Freezing layers ensures they are not trained during the initial fine-tuning process.

In [ ]:
#Freeze the Base Model Layers
for layer in base_model.layers:
    layer.trainable = False

Add custom dense layers to adapt the pre-trained model for the new classification task.

In [ ]:
#Add custom layers for the target task
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) #for binary classification
model = Model(inputs=base_model.input, outputs=x)

#Ensure input shape matched your data generator's output

input_shape = (224, 224, 3) #Assuming you're using RGB images
model.build(input_shape = (None, *input_shape)) #Build the model with an explicit input shape

Choose a lower learning rate to preserve the pre-trained weights and compile the model.

In [ ]:
#Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train') #training data directory
validation_dir = os.path.join(PATH, 'validation') #validation data directory

#BATCH_SIZE = 32
#IMG_SIZE = (160, 160)

68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Load training and validation data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir, #Replace with the path to your training data directory
    target_size=(160, 160), #Adjust if needed
    batch_size=32, #Adjust if needed
    class_mode='binary') #Use categorical
val_data = val_datagen.flow_from_directory(
    validation_dir, #Replace with the path to your validation data directory
    target_size=(160, 160), #Adjust if needed
    batch_size=32, #Adjust if needed
    class_mode='binary') #Use categorical

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
#Training the model
history = model.fit(train_data, epochs=5, validation_data=val_data)